In [1]:
!pip install ipython-autotime
%load_ext autotime

Defaulting to user installation because normal site-packages is not writeable
time: 0 ns (started: 2023-09-05 18:14:52 +03:00)


In [2]:
import torch
print(f'cuda enabled: {torch.backends.cudnn.enabled}')
print(f'cude available: {torch.cuda.is_available()}')

cuda enabled: True
cude available: True
time: 7.75 s (started: 2023-09-05 18:15:41 +03:00)


In [3]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

time: 15 ms (started: 2023-09-05 18:16:00 +03:00)


In [5]:
import os as _os
import pandas as pd
import speech_recognition as sr

audio_file_dir = _os.path.join(
    'data', 'calls')
audio_file_csv = _os.path.join(audio_file_dir, '911_first6sec', '911_metadata_6sec.csv')
df = pd.read_csv(audio_file_csv)

def get_text_from_audio(audio_file_name):

    r = sr.Recognizer()
    dirr, name = audio_file_name.split('/')
    audio_file_path = _os.path.join(audio_file_dir, dirr, name)
    with sr.AudioFile(audio_file_path) as src:
        r.adjust_for_ambient_noise(src, duration=0.5)
        audio = r.record(src)
        return r.recognize_whisper(audio, model='medium')

time: 156 ms (started: 2023-09-05 18:16:59 +03:00)


In [7]:
with torch.device('cuda'):
    df['filename'][:10].apply(get_text_from_audio)

time: 5min 22s (started: 2023-09-05 18:21:51 +03:00)


In [8]:
with torch.device('cpu'):
    df['filename'][:10].apply(get_text_from_audio)

time: 8min (started: 2023-09-05 18:34:29 +03:00)


# Attempt to enable pandarallel

In [3]:
from pandarallel import pandarallel

In [4]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [11]:
text_series = df['filename'][:10].parallel_apply(get_text_from_audio)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 16777216 bytes.